## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [ ]:
# Mounting google drive docs to collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# checking to see if everything is mounted correctly
!ls

drive  sample_data


In [ ]:
# Import and read the charity_data.csv.
application_full = pd.read_csv("/content/drive/MyDrive/BOOTCAMP/Deep-Learning-Challenge/Resources/charity_data.csv")
application_full.head()

## -- 'IS_SUCCESSFUL' seems to be the target for the model, the rest of the columms other than 'EIN' and 'NAME' seem to be the features.

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_full.drop(columns=['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

### -- Categorical data columns APPLICATION_TYPE and CLASSIFICATION both have more than 10 unique values 
### -- should look at them more closely to determine number of data points for each unique value, and which can be binned together

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
appType_vCounts = application_df['APPLICATION_TYPE'].value_counts()
appType_vCounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
appType_cutoff = 500

appType_vCounts_df = pd.DataFrame(appType_vCounts).reset_index()
appType_vCounts_df.columns = ['application_type', 'value_count']
display(appType_vCounts_df)

application_types_to_replace = appType_vCounts_df.loc[appType_vCounts_df['value_count']<appType_cutoff, 'application_type'].to_list()
print(application_types_to_replace)

,application_type,value_count
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']


In [ ]:
# Create a copy of the dataframe for the reduced features
application_df_reduced = application_df.copy()

# Replace target application type data in dataframe
for app in application_types_to_replace:
    application_df_reduced['APPLICATION_TYPE'] = application_df_reduced['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_reduced['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
appCls_vCounts = application_df_reduced['CLASSIFICATION'].value_counts()
appCls_vCounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1570        1
C6100        1
C1728        1
C4500        1
C1732        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
appCls_vCounts.loc[appCls_vCounts >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C1260        3
C0           3
C2710        3
C1234        2
C1256        2
C1246        2
C1267        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
appCls_cutoff = 1000

appCls_vCounts_df = pd.DataFrame(appCls_vCounts).reset_index()
appCls_vCounts_df.columns = ['classification', 'value_count']
display(appCls_vCounts_df)

classifications_to_replace = appCls_vCounts_df.loc[appCls_vCounts_df['value_count']<appCls_cutoff, 'classification'].to_list()
print(classifications_to_replace)

,classification,value_count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C1570,1
67,C6100,1
68,C1728,1
69,C4500,1


['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C2300', 'C7200', 'C1240', 'C8000', 'C7120', 'C1500', 'C6000', 'C1800', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C1720', 'C4100', 'C2400', 'C1257', 'C1600', 'C1260', 'C0', 'C2710', 'C1234', 'C1256', 'C1246', 'C1267', 'C3200', 'C1283', 'C2170', 'C1370', 'C5200', 'C3700', 'C4200', 'C1580', 'C1236', 'C2600', 'C2561', 'C2500', 'C1245', 'C2380', 'C1900', 'C2190', 'C2570', 'C1820', 'C4120', 'C8210', 'C1248', 'C2150', 'C1570', 'C6100', 'C1728', 'C4500', 'C1732']


In [ ]:
# Replace target classification data in reduced features dataframe
for cls in classifications_to_replace:
    application_df_reduced['CLASSIFICATION'] = application_df_reduced['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_reduced['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_reduced_n = pd.get_dummies(application_df_reduced)
application_df_reduced_n

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_reduced_n.drop(columns=['IS_SUCCESSFUL'])
y = application_df_reduced_n['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputFeatures_n = len(X_train_scaled[0])
hiddenNodes_L1 = 80
hiddenNodes_L2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenNodes_L1, activation='relu', input_dim=inputFeatures_n))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenNodes_L2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
cp = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/BOOTCAMP/Deep-Learning-Challenge/Output/weights.h5', save_weights_only=True, periods=5)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7401
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5341 - accuracy: 0.7400
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7408
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7398
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5340 - accuracy: 0.7394
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5339 - accuracy: 0.7399
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5342 - accuracy: 0.7399
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7399
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7395
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5730 - accuracy: 0.7299 - 439ms/epoch - 2ms/step
Loss: 0.5729537606239319, Accuracy: 0.729912519454956


In [ ]:
# Export our model to HDF5 file -- name it AlphabetSoupCharity.h5
nn.save('/content/drive/MyDrive/BOOTCAMP/Deep-Learning-Challenge/Output/AlphabetSoupCharity.h5')